In [ ]:
import os
import cv2
import time
import numpy as np
import import_ipynb
import XInput
from grabscreen import grab_screen
from inputManager import get_input
from IPython.display import clear_output

In [ ]:
# Data load
training_data_file_name = 'data/training_data.npy'
if os.path.isfile(training_data_file_name):
    training_data = list(np.load(training_data_file_name,allow_pickle=True))
else:
    training_data = []
    
gtav_helper_log = open("C:\\Program files\Cheat Engine 7.1\log.txt","r")
objects_cascade = cv2.CascadeClassifier('haarcascade_car.xml')

In [ ]:
tmpTime = time.time()
#######################################
Window_Width = 1920
Window_Height = 835
grabBox = (0,40,Window_Width,Window_Height)
navi_size = (0,835,306,1080)

time_per_screen = 0.15

#######################################    
gtav_helper_log = open("C:\\Program files\Cheat Engine 7.1\log.txt","r")
objects_cascade = cv2.CascadeClassifier('haarcascade_car.xml')

def Proces_Image(Org_Img, LoopTime):
    # Color conversion
    processed_Image = cv2.cvtColor(Org_Img,cv2.COLOR_BGR2RGB)
    # fps Adding
    fps = "FPS: {:5.2f}".format( 1 / LoopTime)
    org = (int(Window_Width*0.8),int(Window_Height*0.05))
    cv2.putText(processed_Image, fps, org, cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,120,0), 2)
    # rectangles drawing over detected objects
    objects = objects_cascade.detectMultiScale(processed_Image, 1.3, 5)
    for (x,y,w,h) in objects:
        cv2.rectangle(processed_Image,(x,y),(x+w,y+h),(255,0,0),5)
    
    return processed_Image

# Collecting data for training loop
cv2.imshow('TST',np.zeros((150,150)))
paused = True
while(True):
    if not paused:
        # Taking screen image
        screen = np.array(grab_screen(region=grabBox))
        screen = cv2.cvtColor(screen,cv2.COLOR_BGR2GRAY)
        screen = cv2.resize(screen, (220, 96))
        # Taking speed and drive data
        gtav_helper_log.seek(0)
        speed = float(gtav_helper_log.read(5))
        forw_back_input = XInput.get_trigger_values(XInput.get_state(0))
        forw_back = (forw_back_input[1] - forw_back_input[0] + 1) / 2
        left_right = (XInput.get_thumb_values(XInput.get_state(0))[0][0] +1) / 2
        
        training_data.append([screen,speed,forw_back,left_right])   
        if (time.time() - tmpTime < time_per_screen):
            time.sleep(time.time() - tmpTime)
        tmpTime=time.time()
        
    # Cheking if pressing pause
    clear_output(wait=True)
    print(paused)
    if 'T' in get_input():
        paused = not paused
        time.sleep(0.5)
    # Closing when q
    if cv2.waitKey(25) & 0xFF == ord('q'):
        np.save(training_data_file_name,training_data)
        cv2.destroyAllWindows()
        break

In [ ]:
# Show slideshow
i=0
len(training_data)
while(i<len(training_data)):
    cv2.imshow('TST', training_data[i][0])
    i+=1
    if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break
cv2.destroyAllWindows()